**Task 2**

In [5]:
from google.colab import drive
import numpy as np
import pandas as pd
import os
import glob
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

Check Cuda Availability

In [6]:
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Mount google drive

In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


Define data path

In [8]:
class_priority_path = '/content/drive/My Drive/data/Priority'
class_stop_path = '/content/drive/My Drive/data/Stop'

In [9]:
priority_images = [image for image in os.listdir(class_priority_path) if image.endswith('.jpg')]
stop_images = [image for image in os.listdir(class_stop_path) if image.endswith('.jpg')]
labeled_priority_images_full_path = [(os.path.join(class_priority_path, image), 1) for image in priority_images]
labeled_stop_images_full_path = [(os.path.join(class_stop_path, image), 0) for image in stop_images]
data = labeled_priority_images_full_path + labeled_stop_images_full_path

Load Data to x and y (features and label)

In [10]:
X_array = []
y_array = []

for item in data:
  x_image = cv2.imread(item[0])
  x_image = cv2.resize(x_image, (224, 224))
  X_array.append(x_image)
  y_image = item[1]
  y_array.append(y_image)

X_array = np.array(X_array)
y_array = np.array(y_array)

Convert Data to Tensor

In [11]:
X = torch.tensor(X_array, dtype=torch.float32)
y = torch.tensor(y_array, dtype=torch.int64)

Train-Test Split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=122)
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

Load ResNet18

In [13]:
baseModel = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
baseModel.fc = nn.Linear(512, 512)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 90.6MB/s]


Create Model

In [14]:
hidden_units = 128
output_units = 2
activation_function = nn.Sigmoid()

In [15]:
headModel = nn.AdaptiveAvgPool2d((1,1))
hidden_layer = nn.Linear(512, hidden_units)
dropout_layer = nn.Dropout(p=0.5)
output_layer = nn.Linear(hidden_units, output_units)
model_resnet18_classification = nn.Sequential(baseModel, hidden_layer, activation_function, output_layer)
model_resnet18_classification.to(device)

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

Train Data

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,train_size=0.9, test_size=0.1, random_state=42)

epochs = 30
batch_size = 16
loss_function = nn.CrossEntropyLoss()
num_batches_train = (len(X_train) + batch_size - 1) // batch_size
num_batches_val = (len(X_val) + batch_size - 1) // batch_size

optimizer = optim.Adam(model_resnet18_classification.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=True)

for epoch in range(epochs):

  model_resnet18_classification.train().to(device)
  train_loss = 0

  for i in range(num_batches_train):
    batch_inputs = X_train[i * batch_size:(i + 1) * batch_size].permute(0, 3, 1, 2) # (N, C, H, W, C, N)
    batch_labels = y_train[i * batch_size:(i + 1) * batch_size]

    optimizer.zero_grad()
    outputs = model_resnet18_classification(batch_inputs)
    loss = loss_function(outputs, batch_labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

  epoch_loss = train_loss / num_batches_train

  model_resnet18_classification.eval().to(device)
  val_loss = 0

  with torch.no_grad():
    for j in range(num_batches_val):
      batch_inputs = X_val[j * batch_size:(j + 1) * batch_size].permute(0, 3, 1, 2)
      batch_labels = y_val[j * batch_size:(j + 1) * batch_size]

      outputs = model_resnet18_classification(batch_inputs)
      loss = loss_function(outputs, batch_labels)
      val_loss += loss.item()

    val_loss /= num_batches_val
    scheduler.step(val_loss)

  epoch_train_loss = train_loss / num_batches_train

  print(f'epoch {epoch+1} Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


epoch 1 Train Loss: 5.779869, Val Loss: 0.044204
epoch 2 Train Loss: 2.251409, Val Loss: 0.241062
epoch 3 Train Loss: 1.874122, Val Loss: 0.670971
epoch 4 Train Loss: 0.471400, Val Loss: 0.014633
epoch 5 Train Loss: 0.207992, Val Loss: 0.008807
epoch 6 Train Loss: 0.141926, Val Loss: 0.006544
epoch 7 Train Loss: 0.108880, Val Loss: 0.005261
epoch 8 Train Loss: 0.088598, Val Loss: 0.004376
epoch 9 Train Loss: 0.074289, Val Loss: 0.003722
epoch 10 Train Loss: 0.063569, Val Loss: 0.003218
epoch 11 Train Loss: 0.055239, Val Loss: 0.002820
epoch 12 Train Loss: 0.048594, Val Loss: 0.002497
epoch 13 Train Loss: 0.043184, Val Loss: 0.002232
epoch 14 Train Loss: 0.038706, Val Loss: 0.002010
epoch 15 Train Loss: 0.034948, Val Loss: 0.001822
epoch 16 Train Loss: 0.031755, Val Loss: 0.001662
epoch 17 Train Loss: 0.029016, Val Loss: 0.001523
epoch 18 Train Loss: 0.026644, Val Loss: 0.001402
epoch 19 Train Loss: 0.024573, Val Loss: 0.001297
epoch 20 Train Loss: 0.022753, Val Loss: 0.001203
epoch 21 

Save Model

In [17]:
torch.save(model_resnet18_classification.state_dict(), 'model_resnet18_classification.pth')

Load Model

In [18]:
baseModel = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
baseModel.fc = torch.nn.Linear(512, 512)
headModel = torch.nn.AdaptiveAvgPool2d((1,1))
hidden_layer = torch.nn.Linear(512, 128)
dropout_layer = torch.nn.Dropout(p=0.5)
output_layer = torch.nn.Linear(128, 2)
model_resnet18_classification = torch.nn.Sequential(baseModel, hidden_layer, torch.nn.Sigmoid(), output_layer)
model_resnet18_classification = model_resnet18_classification.to(device).eval()
model_resnet18_classification.load_state_dict(torch.load('model_resnet18_classification.pth'))

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


<All keys matched successfully>

Model Evaluation

In [19]:
model_resnet18_classification.eval().to(device)

num_batches_test = (len(X_test) + batch_size - 1) // batch_size
actuals = []
predictions = []

with torch.no_grad():
  for i in range(num_batches_test):
    batch_inputs_test = X_test[i * batch_size:(i + 1) * batch_size].permute(0, 3, 1, 2)
    batch_labels_test = y_test[i * batch_size:(i + 1) * batch_size]

    outputs = model_resnet18_classification(batch_inputs_test)
    _, predicted_labels = torch.max(outputs, 1)
    actuals.append(batch_labels_test.cpu().numpy())
    predictions.append(outputs.cpu().numpy())

actuals = np.concatenate(actuals, axis=0)
predictions = np.concatenate(predictions, axis=0)

accuracy = accuracy_score(actuals, predictions)
precision = precision_score(actuals, predictions, average='weighted')
recall = recall_score(actuals, predictions, average='weighted')
f1 = f1_score(actuals, predictions, average='weighted')

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets